# Colab Inference (code-only walkthrough)

This notebook mirrors the Gradio launcher but runs everything in plain Python cells so you can observe every inference step and timing. It downloads the helper `gradio_app.py`, loads the YOLO model, tiles the image, runs batch predictions, fuses detections, and visualises the output without starting a Gradio UI.


## 1) Install runtime dependencies

Install the exact packages expected by the helper module. Comment out lines for packages that are already available in your runtime.


In [ ]:
# Install minimal runtime dependencies. Comment out packages you already have in your runtime.
!pip install -q ultralytics shapely pillow torch matplotlib requests


## 2) Configure download sources and runtime options

Update the URLs below if you are hosting the weights or helper in a different location. The device and batch size are derived from your runtime capabilities so you can see when GPU is unavailable.


In [ ]:
import requests
from pathlib import Path
import torch

# ---------------------------------------------------------------------------
# Configure your download locations here
# ---------------------------------------------------------------------------
REMOTE_BASE = 'https://collembot.ch/colab-resources/'  # hosting root with model, helper, and example image
MODEL_URL = REMOTE_BASE + 'collembot-2025_12-yolo11x-seg.pt'
HELPER_URL = REMOTE_BASE + 'gradio_app.py'
EXAMPLE_URL = REMOTE_BASE + 'example.jpg'

ROOT = Path('/content')
ROOT.mkdir(exist_ok=True)
weights_path = ROOT / 'collembot-2025_12-yolo11x-seg.pt'
helper_path = ROOT / 'gradio_app.py'
example_path = ROOT / 'example.jpg'

USE_GPU = torch.cuda.is_available()
PREFERRED_DEVICE = 'cuda:0' if USE_GPU else 'cpu'
USE_HALF = False
BATCH_SIZE = 12 if USE_GPU else 4

if USE_GPU:
    name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f'✅ GPU detected: {name}')
else:
    print('⚠️ No GPU detected. In Colab, choose GPU via Runtime > Change runtime type for faster inference.')
print(f'Using device={PREFERRED_DEVICE}, half={USE_HALF}, batch={BATCH_SIZE}')


## 3) Download helper, weights, and example image

The helper is reused from the Gradio notebook, but we will call its functions directly. Downloads are forced each run to keep the notebook reproducible.


In [ ]:
def download(url: str, dest: Path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    dest.parent.mkdir(parents=True, exist_ok=True)
    with open(dest, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f'Downloaded {url} -> {dest}')

# Download the helper, weights, and example image. They will overwrite existing copies
# to keep the notebook reproducible.
download(HELPER_URL, helper_path)
download(MODEL_URL, weights_path)
download(EXAMPLE_URL, example_path)


## 4) Import the helper module

The helper exposes the same tiling, inference, fusion, and visualisation utilities used by the Gradio app.


In [ ]:
import importlib.util

spec = importlib.util.spec_from_file_location('gradio_app', helper_path)
gradio_app = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gradio_app)
print(f'Loaded helper from {helper_path}')


## 5) Load the YOLO model

Use the helper's `load_model` to initialise the network on the chosen device. Timing is captured so you can tell how long model loading takes.


In [ ]:
import time

load_start = time.time()
model, device, half_flag = gradio_app.load_model(
    str(weights_path),
    preferred_device=PREFERRED_DEVICE,
    use_half=USE_HALF,
)
load_end = time.time()
print(f'Model loaded on {device} (half={half_flag}) in {load_end - load_start:.2f}s')


## 6) Load an image to analyse

You can replace `example_path` with your own upload (e.g., via the Colab file sidebar). The filename is stored on the PIL image so circle detection can reuse the path.


In [ ]:
from PIL import Image

image_path = example_path  # swap this for your own path if desired
image = Image.open(image_path).convert('RGB')
image.filename = str(image_path)
print(f'Loaded image {image_path} with size {image.size}')


## 7) Tile the image

Run the same tiling strategy as the Gradio app (original and shifted grids). This cell reports how many tiles are produced and previews their sizes.


In [ ]:
tile_start = time.time()
tiles = gradio_app.build_tiles(image, max_workers=gradio_app.CPU_WORKERS)
tile_end = time.time()
print(f'Generated {len(tiles)} tiles in {tile_end - tile_start:.2f}s')
if tiles:
    sample = tiles[0]
    print(f"First tile tag={sample['tag']}, offset={sample['offset']}, size={sample['tile'].size}")


## 8) Run YOLO predictions on the tiles

The helper's `run_model_on_tiles` is called directly with a simple progress logger so you can see batch-level timing and GPU/CPU usage.


In [ ]:
def log_progress(frac: float, desc: str = ''):
    percent = int(frac * 100)
    print(f'[{percent:3d}%] {desc}')

infer_start = time.time()
raw_polys = gradio_app.run_model_on_tiles(
    model,
    device,
    tiles,
    batch_size=BATCH_SIZE,
    use_half=half_flag,
    progress=log_progress,
    progress_range=(0.0, 0.9),
)
infer_end = time.time()
print(f'Collected {len(raw_polys)} raw polygons in {infer_end - infer_start:.2f}s')


## 9) Fuse overlapping detections

Graph-cut fusion merges overlapping polygons across tiles. This step can be a bottleneck for large images; the timing helps you gauge its cost.


In [ ]:
fuse_start = time.time()
fused = gradio_app.fuse_graphcut(raw_polys, iou_th=gradio_app.BEST_IOU, alpha=gradio_app.ALPHA)
fuse_end = time.time()
print(f'Fused to {len(fused)} polygons in {fuse_end - fuse_start:.2f}s')


## 10) Filter detections to the main circle

A lightweight circle detector is reused to discard polygons outside the dish area (when found). Both the circle and filtered count are printed.


In [ ]:
circle = gradio_app.find_main_circle(image)
filtered = gradio_app.filter_polygons_by_circle(fused, circle)
print(f'Circle: {circle}')
print(f'Filtered polygons: {len(filtered)} (from {len(fused)})')


## 11) Visualise and export results

Draw the polygons and optional circle onto the original image. The preview is shown inline and also saved for download.


In [ ]:
from IPython.display import display

viz = gradio_app.draw_polygons(image, filtered, circle)
output_path = ROOT / 'detections.png'
viz.save(output_path)
print(f'Visualisation saved to {output_path}')
display(viz)
